<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/2DCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive
ls: cannot access '/content/drive/mydrive': No such file or directory


In [2]:
%matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow

TRAIN_PATH = "train"
TEST_PATH ="test"

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Questo commento serve per provare il funzionamento dei commit con colab

# **Dataset Loading**

Unzip del dataset (Temporaneamente da caricare da file)

In [4]:
import zipfile


zip_ref = zipfile.ZipFile('fer2013.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

Associazione path delle immagini - classe delle immagini

In [18]:
import os
class_dir=os.listdir(TRAIN_PATH+'/')
folder_names = class_dir
label_dict = {folder_names[i]:i for i in range(len(folder_names))}
num_classes=len(label_dict)
train_image_filenames = []
train_labels = []

# iterate through each folder and collect filenames and labels
for folder_name in folder_names:
    folder_path = os.path.join(TRAIN_PATH, folder_name)
    for filename in os.listdir(folder_path):
        train_image_filenames.append(os.path.join(TRAIN_PATH+"/"+folder_name, filename))
        train_labels.append(label_dict[folder_name])

# create pandas dataframe
train_df = pd.DataFrame({'filename': train_image_filenames, 'emotion': train_labels})

print(len(train_df))


test_image_filenames = []
test_labels = []
class_dir=os.listdir(TEST_PATH+'/')
for folder_name in folder_names:
    folder_path = os.path.join(TEST_PATH, folder_name)
    for filename in os.listdir(folder_path):
        test_image_filenames.append(os.path.join(TEST_PATH+"/"+folder_name, filename))
        test_labels.append(label_dict[folder_name])

test_df = pd.DataFrame({'filename': test_image_filenames, 'emotion': test_labels})

28709


Caricamento matrici delle immagini

In [19]:
train_df['img_as_matrix'] = train_df['filename'].apply(lambda path: cv2.imread(path))
test_df['img_as_matrix'] = test_df['filename'].apply(lambda path: cv2.imread(path))


Creazione del dataset, estendendo la classe dataset di PyTorch

In [20]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data.iloc[index]['img_as_matrix'] 
        label = self.data.iloc[index]['emotion'] 

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

Creazione dei Loader

In [21]:
train_df.drop(columns=["filename"], inplace=True)
test_df.drop(columns=["filename"], inplace=True)
train_df.at[0,"img_as_matrix"]
train_dataset = CustomDataset(train_df, transform=transforms.ToTensor())
test_dataset = CustomDataset(test_df, transform=transforms.ToTensor())

batch_size = 32  

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **Rete convolutiva**

In [34]:
def accuracy(preds, labels):
  _, predicted = torch.max(preds, 1) 
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc;


In [23]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        
        x = self.fc3(x)
        
        return x


In [37]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [38]:
n_total_steps = len(train_loader)
num_epochs = 20
for epoch in range(num_epochs):

    running_loss = 0.0
    running_acc = 0.0

    for i, (images, labels) in enumerate(train_loader):
       
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)

      loss = criterion(outputs, labels)
      _, predicted = torch.max(outputs, 1)

      acc = accuracy(outputs, labels)

      optimizer.zero_grad()      
      loss.backward()            
      optimizer.step()  
      running_loss += loss.item()    
      running_acc += acc.item()

    print (f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Acc: {running_acc/len(train_loader):.4f}')

print('Finished Training')

PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [0/20], Loss: 1.8403, Acc: 24.7016
Epoch [1/20], Loss: 1.8067, Acc: 25.1258
Epoch [2/20], Loss: 1.7961, Acc: 25.0958
Epoch [3/20], Loss: 1.7823, Acc: 25.8608
Epoch [4/20], Loss: 1.7609, Acc: 28.1470
Epoch [5/20], Loss: 1.7238, Acc: 30.9599
Epoch [6/20], Loss: 1.6807, Acc: 33.7884
Epoch [7/20], Loss: 1.6446, Acc: 35.9109
Epoch [8/20], Loss: 1.6143, Acc: 37.4376
Epoch [9/20], Loss: 1.5825, Acc: 38.7416
Epoch [10/20], Loss: 1.5517, Acc: 40.3541
Epoch [11/20], Loss: 1.5213, Acc: 41.5089
Epoch [12/20], Loss: 1.4905, Acc: 42.9065
Epoch [13/20], Loss: 1.4615, Acc: 43.7962
Epoch [14/20], Loss: 1.4303, Acc: 45.1726
Epoch [15/20], Loss: 1.4006, Acc: 46.3664
Epoch [16/20], Loss: 1.3677, Acc: 47.8998
Epoch [17/20], Loss: 1.3374, Acc: 49.1347
Epoch [18/20], Loss: 1.3063, Acc: 50.4220
Epoch [19/20], Loss: 1.2727, Acc: 51.9477
Finished Training


In [40]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(7)]
    n_class_samples = [0 for i in range(7)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
       
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(labels.size(0)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(7):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {i}: {round(acc, 2)} %')

Accuracy of the network: 45.945945945945944 %
Accuracy of 0: 61.37 %
Accuracy of 1: 87.49 %
Accuracy of 2: 18.58 %
Accuracy of 3: 12.99 %
Accuracy of 4: 47.53 %
Accuracy of 5: 27.19 %
Accuracy of 6: 0.0 %
